In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df= pd.read_csv('rav_framingham.csv')
df.head()

In [ ]:
df1 = pd.read_csv('diabetes_.csv')
df1.head()

In [ ]:
df1 = df1.astype(float)
df1.head()

In [ ]:
df1['Glucose'] = df1['Glucose'].replace(0, np.median(df1['Glucose']))
df1['BloodPressure'] = df1['BloodPressure'].replace(0, np.median(df1['BloodPressure']))
df1['SkinThickness'] = df1['SkinThickness'].replace(0, np.median(df1['SkinThickness']))
df1['Insulin'] = df1['Insulin'].replace(0, np.median(df1['Insulin']))
df1['BMI'] = df1['BMI'].replace(0, np.median(df1['BMI']))

In [ ]:
Xd = df1.drop('Outcome', axis=1)
yd = df1['Outcome']

In [ ]:
from sklearn.model_selection import train_test_split
Xd_train, Xd_test, yd_train, yd_test = train_test_split(Xd, yd, test_size=0.2, random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(Xd_train)
X_test_scaled = scaler.transform(Xd_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
# Random Forest without scaling or resampling
rfd = RandomForestClassifier(random_state=42)
rfd.fit(Xd_train, yd_train)
predictions = rfd.predict(Xd_test)
print("\nRandom Forest - Original Data")
print("Classification Report:")
print(classification_report(yd_test, predictions))
print("Confusion Matrix:")
print(confusion_matrix(yd_test, predictions))
# Random Forest with scaling
rfd_scaled = RandomForestClassifier(random_state=42)
rfd_scaled.fit(X_train_scaled, yd_train)
predictions_scaled = rfd_scaled.predict(X_test_scaled)
print("\nRandom Forest - Scaled Data")
print("Classification Report:")
print(classification_report(yd_test, predictions_scaled))
print("Confusion Matrix:")
print(confusion_matrix(yd_test, predictions_scaled))

In [ ]:
df.keys()


In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
# drop rows with NAN/Missing values
df.dropna(axis = 0, inplace = True)

print(df.shape)
print(df.isna().sum())

In [ ]:
count=df.duplicated().sum()
print("Number of Duplicate values: ",count)

In [ ]:
df.drop(columns=['education'] , inplace=True)
df.head()

In [ ]:
# rename a feature
df=df.rename(columns={'male':'gender'})

In [ ]:
x = df.iloc[:,0:14]
y = df['TenYearCHD']

In [ ]:
x.head()


In [ ]:
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=21)
X_train2=X_train
X_test2=X_test

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# Random Forest without scaling or resampling
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)
print("\nRandom Forest - Original Data")
print("Classification Report:")
print(classification_report(y_test, predictions))
print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))



In [ ]:
import sys
import sqlite3
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QLineEdit, QPushButton, QMessageBox
from PyQt5.QtGui import (QPixmap)
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton,QWidget,QVBoxLayout
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
from PyQt5 import QtGui, QtCore
import pandas as pd
from PyQt5.QtWidgets import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
import sys
import sqlite3
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QLineEdit, QPushButton, QMessageBox
from PyQt5.QtGui import (QPixmap)

class LoginWindow(QWidget):
    
    def __init__(self, parent=None):
        super().__init__()
        self.parent = parent
        self.conn = sqlite3.connect('users.db')
        self.cursor = self.conn.cursor()

        self.init_ui()

    def init_ui(self):
        layout = QVBoxLayout()

        self.login_username = QLineEdit()
        self.login_password = QLineEdit()
        self.login_password.setEchoMode(QLineEdit.Password)

        layout.addWidget(QLabel("Login"))
        layout.addWidget(QLabel("Username:"))
        layout.addWidget(self.login_username)
        layout.addWidget(QLabel("Password:"))
        layout.addWidget(self.login_password)

        login_button = QPushButton("Login")
        login_button.clicked.connect(self.login)
        layout.addWidget(login_button)

        self.setLayout(layout)
        self.setWindowTitle("Login")

    def login(self):
        username = self.login_username.text()
        password = self.login_password.text()

        self.cursor.execute('SELECT * FROM users WHERE username = ? AND password = ?', (username, password))
        user = self.cursor.fetchone()

        if user:
            self.parent=ParentGUI()
            self.parent.show()
            
        else:
            QMessageBox.warning(self, "Error", "Invalid username or password")


class RegisterWindow(QWidget):
    def __init__(self, parent=None):
        super().__init__()
        self.parent = parent
        self.conn = sqlite3.connect('users.db')
        self.cursor = self.conn.cursor()
        
        self.init_ui()

    def init_ui(self):
        layout = QVBoxLayout()

        self.register_username = QLineEdit()
        self.register_password = QLineEdit()
        self.register_password.setEchoMode(QLineEdit.Password)

        layout.addWidget(QLabel("Register"))
        layout.addWidget(QLabel("Username:"))
        layout.addWidget(self.register_username)
        layout.addWidget(QLabel("Password:"))
        layout.addWidget(self.register_password)

        register_button = QPushButton("Register")
        register_button.clicked.connect(self.register)
        layout.addWidget(register_button)

        self.setLayout(layout)
        self.setWindowTitle("Register")

    def register(self):
        username = self.register_username.text()
        password = self.register_password.text()

        try:
            self.cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, password))
            self.conn.commit()
            QMessageBox.information(self, "Success", "Registration Successful!")
        except sqlite3.IntegrityError:
            QMessageBox.warning(self, "Error", "Username already exists")


class MainWindow(QWidget):
    def __init__(self):
        super().__init__()

        self.login_window = None
        self.register_window = None
        self.create_table()
        self.init_ui()
        
    def create_table(self):
        conn = sqlite3.connect('users.db')
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS users (
                id INTEGER PRIMARY KEY,
                username TEXT NOT NULL,
                password TEXT NOT NULL
            )
        ''')
        conn.commit()
        conn.close()
    
    

    def init_ui(self):
        layout = QVBoxLayout()

        login_button = QPushButton("Login")
        login_button.setStyleSheet(
            "QPushButton {"
            "   background-color: #4CAF50;"
            "   border: none;"
            "   color: white;"
            "   padding: 10px 20px;"
            "   text-align: center;"
            "   text-decoration: none;"
            "   display: inline-block;"
            "   font-size: 16px;"
            "   margin: 4px 2px;"
            "   cursor: pointer;"
            "   border-radius: 5px;"
            "   min-width: 150px;"  
            "   max-width: 50%;"  
            "   margin: 0 auto;"   
            "}"
            "QPushButton:hover {"
            "   background-color: #45a049;"
            "}"
        )
        login_button.setFixedSize(180, 60)
        login_button.clicked.connect(self.show_login_window)
        layout.addWidget(login_button)

        register_button = QPushButton("Register")
        register_button.setStyleSheet(
            "QPushButton {"
            "   background-color: #4CAF50;"
            "   border: none;"
            "   color: white;"
            "   padding: 10px 20px;"
            "   text-align: center;"
            "   text-decoration: none;"
            "   display: inline-block;"
            "   font-size: 16px;"
            "   margin: 4px 2px;"
            "   cursor: pointer;"
            "   border-radius: 5px;"
            "   min-width: 150px;"  
            "   max-width: 50%;"  
            "   margin: 0 auto;"   
            "}"
            "QPushButton:hover {"
            "   background-color: #45a049;"
            "}"
        )
        register_button.setFixedSize(180, 60)
        register_button.clicked.connect(self.show_register_window)
        
      
        
        layout.addWidget(register_button)
        
        
        self.setLayout(layout)
        self.setWindowTitle("Main Window")

    def show_login_window(self):
        self.login_window = LoginWindow(parent=self)
        self.login_window.show()

    def show_register_window(self):
        self.register_window = RegisterWindow(parent=self)
        self.register_window.show()


class DiabetesGUI(QMainWindow):
    
    def __init__(self):
        super().__init__()
        self.title = "Diabetes Prediction"
        self.top = 300
        self.left = 600
        self.width = 600
        self.height = 600
        self.iconName = "C:/Users/user/Documents/pythonprog/ML/MLGUI/assets/python.png"
        self.initUI()
        
        
    def initUI(self):               
        
        self.setWindowTitle(self.title)
        self.setWindowIcon(QtGui.QIcon(self.iconName))
        self.setGeometry(self.left, self.top, self.width, self.height)

        self.createInputFields()
        self.createPredictionButton()
        self.show()
        
        self.svmButton = self.createButton("Run",self.runModelPrediction,330, 240,60, 30)
        
        self.show()
    def createPredictionButton(self):
        # Create a button for prediction
        self.predict_button = self.createButton("Predict", self.runModelPrediction, 200, 400, 100, 30)

    def createButton(self, text, function, x, y, width, height):
        # Function to create buttons
        button = QPushButton(text, self)
        button.setGeometry(QtCore.QRect(x, y, width, height))
        button.clicked.connect(function)
        return button
    
    def createInputFields(self):

        self.preg_label = QLabel("Preg:", self)
        self.preg_label.setGeometry(QtCore.QRect(40, 50, 80, 30))
        
        self.preg_lineEdit = QLineEdit(self)
        self.preg_lineEdit.setGeometry(QtCore.QRect(150, 50, 100, 30))
        
        self.gluc_label = QLabel("Glucose:", self)
        self.gluc_label.setGeometry(QtCore.QRect(40, 90, 100, 30))
        
        self.gluc_lineEdit = QLineEdit(self)
        self.gluc_lineEdit.setGeometry(QtCore.QRect(150, 90, 100, 30))
        
        self.BP_label = QLabel("BP:", self)
        self.BP_label.setGeometry(QtCore.QRect(40, 130, 80, 30))
        
        self.BP_lineEdit = QLineEdit(self)
        self.BP_lineEdit.setGeometry(QtCore.QRect(150, 130, 100, 30))

        self.SkinT_label = QLabel("SkinThickness:", self)
        self.SkinT_label.setGeometry(QtCore.QRect(40, 170, 80, 30))
        
        self.SkinT_lineEdit = QLineEdit(self)
        self.SkinT_lineEdit.setGeometry(QtCore.QRect(150, 170, 100, 30))
        
        self.Insulin_label = QLabel("Insulin:", self)
        self.Insulin_label.setGeometry(QtCore.QRect(260, 50, 100, 30))
        
        self.Insulin_lineEdit = QLineEdit(self)
        self.Insulin_lineEdit.setGeometry(QtCore.QRect(370, 50, 100, 30))

        self.BMI_label = QLabel("BMI:", self)
        self.BMI_label.setGeometry(QtCore.QRect(260, 90, 80, 30))
        
        self.BMI_lineEdit = QLineEdit(self)
        self.BMI_lineEdit.setGeometry(QtCore.QRect(370, 90, 100, 30))

        self.Diabetes_label = QLabel("DPF", self)
        self.Diabetes_label.setGeometry(QtCore.QRect(260, 130, 80, 30))
        
        self.Diabetes_lineEdit = QLineEdit(self)
        self.Diabetes_lineEdit.setGeometry(QtCore.QRect(370, 130, 100, 30))
        
        self.age_label = QLabel("Age", self)
        self.age_label.setGeometry(QtCore.QRect(260, 170, 100, 30))
        
        self.age_lineEdit = QLineEdit(self)
        self.age_lineEdit.setGeometry(QtCore.QRect(370, 170, 100, 30))


        self.predict_button = self.createButton("Predict", self.runModelPrediction, 200, 400, 100, 30)
        
        
    def setDefault(self):
        # self.fileName = ""
        self.splitSize = 20
        self.regParam = 1.0
        self.kernelType = 'rbf'
        self.degree = 3
        self.tol = 0.001
        
    
    def drawBrowser(self):
        self.centralwidget = QWidget(self) 
        self.csv_label = QLabel(self.centralwidget) 
        self.csv_label.setGeometry(QtCore.QRect(10, 10, 80, 30))
        self.csv_label.setText("csv file: ")
        
        self.csv_lineEdit = QLineEdit(self)
        self.csv_lineEdit.setGeometry(QtCore.QRect(90,10,300,30))
        self.svmButton = self.createButton("Browse",self.getFileName,330, 50,60, 30)
        

    def runModelPrediction(self):
        # Assuming you have loaded your trained model (rf_model) beforehand
        # Get user input data from GUI fields
        input_data = {
            "Pregnancies": int(self.preg_lineEdit.text()),
            "Glucose": int(self.gluc_lineEdit.text()),
            "BloodPressure": int(self.BP_lineEdit.text()),
            "SkinThickness": int(self.SkinT_lineEdit.text()),
            "Insulin": int(self.Insulin_lineEdit.text()),
            "BMI": float(self.BMI_lineEdit.text()),
            "DiabetesPedigreeFunction": float(self.Diabetes_lineEdit.text()),
            "Age": int(self.age_lineEdit.text()),
            
            # Extract data from other fields similarly
            # ...
        }
        
        # Convert input data to a DataFrame
        input_df = pd.DataFrame([input_data])
        
        # Assuming rf_model is your trained Random Forest model
        predicted_result = self.predictOutcome(input_df,rfd)
        
        # Display the prediction result in a message box
        if predicted_result[0] == 1:
            QMessageBox.about(self, "Prediction Result", "The model predicts a high risk of Diabetes.")
        else:
            QMessageBox.about(self, "Prediction Result", "The model predicts a low risk of Diabetes.")



    def predictOutcome(self, input_data,model):
        # Load your trained Random Forest model
        # Replace this with your actual trained model
        predicted_outcome = model.predict(input_data)
        return predicted_outcome
        

class HeartGUI(QMainWindow):
    
    def __init__(self):
        super().__init__()
        self.title = "Heart Risk Prediction"
        self.top = 300
        self.left = 600
        self.width = 600
        self.height = 600
        self.iconName = "C:/Users/user/Documents/pythonprog/ML/MLGUI/assets/python.png"
        self.initUI()
        
        
    def initUI(self):               
        
        self.setWindowTitle(self.title)
        self.setWindowIcon(QtGui.QIcon(self.iconName))
        self.setGeometry(self.left, self.top, self.width, self.height)

        self.createInputFields()
        self.createPredictionButton()
        self.show()
        
        self.svmButton = self.createButton("Run",self.runModelPrediction,330, 240,60, 30)
        
        self.show()
    def createPredictionButton(self):
        # Create a button for prediction
        self.predict_button = self.createButton("Predict", self.runModelPrediction, 200, 400, 100, 30)

    def createButton(self, text, function, x, y, width, height):
        # Function to create buttons
        button = QPushButton(text, self)
        button.setGeometry(QtCore.QRect(x, y, width, height))
        button.clicked.connect(function)
        return button
    
    def createInputFields(self):
        # Create input fields for user data
        # Example: Creating fields for gender, age, etc.
        self.gender_label = QLabel("Gender:", self)
        self.gender_label.setGeometry(QtCore.QRect(40, 50, 80, 30))
        
        self.gender_lineEdit = QLineEdit(self)
        self.gender_lineEdit.setGeometry(QtCore.QRect(150, 50, 100, 30))

        self.age_label = QLabel("Age:", self)
        self.age_label.setGeometry(QtCore.QRect(40, 90, 80, 30))
        
        self.age_lineEdit = QLineEdit(self)
        self.age_lineEdit.setGeometry(QtCore.QRect(150, 90, 100, 30))
        
        self.smoker_label = QLabel("Smoker (1/0):", self)
        self.smoker_label.setGeometry(QtCore.QRect(40, 130, 100, 30))
        
        self.smoker_lineEdit = QLineEdit(self)
        self.smoker_lineEdit.setGeometry(QtCore.QRect(150, 130, 100, 30))
        
        self.cigspd_label = QLabel("Cigs Per Day", self)
        self.cigspd_label.setGeometry(QtCore.QRect(40, 170, 80, 30))
        
        self.cigspd_lineEdit = QLineEdit(self)
        self.cigspd_lineEdit.setGeometry(QtCore.QRect(150, 170, 100, 30))

        self.BPmeds_label = QLabel("BPMeds", self)
        self.BPmeds_label.setGeometry(QtCore.QRect(40, 210, 80, 30))
        
        self.BPmeds_lineEdit = QLineEdit(self)
        self.BPmeds_lineEdit.setGeometry(QtCore.QRect(150, 210, 100, 30))
        
        self.Pstrokes_label = QLabel("Prevalent Stroke", self)
        self.Pstrokes_label.setGeometry(QtCore.QRect(40, 250, 100, 30))
        
        self.Pstrokes_lineEdit = QLineEdit(self)
        self.Pstrokes_lineEdit.setGeometry(QtCore.QRect(150, 250, 100, 30))

        self.Phyp_label = QLabel("Prevalent HYP", self)
        self.Phyp_label.setGeometry(QtCore.QRect(40, 290, 80, 30))
        
        self.Phyp_lineEdit = QLineEdit(self)
        self.Phyp_lineEdit.setGeometry(QtCore.QRect(150, 290, 100, 30))

        self.Diabetes_label = QLabel("Diabetes", self)
        self.Diabetes_label.setGeometry(QtCore.QRect(260, 50, 80, 30))
        
        self.Diabetes_lineEdit = QLineEdit(self)
        self.Diabetes_lineEdit.setGeometry(QtCore.QRect(370, 50, 100, 30))
        
        self.Totchol_label = QLabel("Tot Chol", self)
        self.Totchol_label.setGeometry(QtCore.QRect(260, 90, 100, 30))
        
        self.Totchol_lineEdit = QLineEdit(self)
        self.Totchol_lineEdit.setGeometry(QtCore.QRect(370, 90, 100, 30))

        self.sysbp_label = QLabel("Sys BP", self)
        self.sysbp_label.setGeometry(QtCore.QRect(260, 130, 80, 30))
        
        self.sysbp_lineEdit = QLineEdit(self)
        self.sysbp_lineEdit.setGeometry(QtCore.QRect(370, 130, 100, 30))

        self.diabp_label = QLabel("Dia BP", self)
        self.diabp_label.setGeometry(QtCore.QRect(260, 170, 100, 30))
        
        self.diabp_lineEdit = QLineEdit(self)
        self.diabp_lineEdit.setGeometry(QtCore.QRect(370, 170, 100, 30))

        self.BMI_label = QLabel("BMI", self)
        self.BMI_label.setGeometry(QtCore.QRect(260, 210, 80, 30))
        
        self.BMI_lineEdit = QLineEdit(self)
        self.BMI_lineEdit.setGeometry(QtCore.QRect(370, 210, 100, 30))

        self.hrate_label = QLabel("Heart Rate", self)
        self.hrate_label.setGeometry(QtCore.QRect(260, 250, 100, 30))
        
        self.hrate_lineEdit = QLineEdit(self)
        self.hrate_lineEdit.setGeometry(QtCore.QRect(370, 250, 100, 30))

        self.gluco_label = QLabel("Glucose", self)
        self.gluco_label.setGeometry(QtCore.QRect(260, 290, 80, 30))
        
        self.gluco_lineEdit = QLineEdit(self)
        self.gluco_lineEdit.setGeometry(QtCore.QRect(370, 290, 100, 30))


        self.predict_button = self.createButton("Predict", self.runModelPrediction, 200, 400, 100, 30)
        
        
    def setDefault(self):
        # self.fileName = ""
        self.splitSize = 20
        self.regParam = 1.0
        self.kernelType = 'rbf'
        self.degree = 3
        self.tol = 0.001
        
    
    def drawBrowser(self):
        self.centralwidget = QWidget(self) 
        self.csv_label = QLabel(self.centralwidget) 
        self.csv_label.setGeometry(QtCore.QRect(10, 10, 80, 30))
        self.csv_label.setText("csv file: ")
        
        self.csv_lineEdit = QLineEdit(self)
        self.csv_lineEdit.setGeometry(QtCore.QRect(90,10,300,30))
        self.svmButton = self.createButton("Browse",self.getFileName,330, 50,60, 30)
        

    def runModelPrediction(self):
        # Assuming you have loaded your trained model (rf_model) beforehand
        # Get user input data from GUI fields
        input_data = {
            "gender": int(self.gender_lineEdit.text()),
            "age": int(self.age_lineEdit.text()),
            "currentSmoker": int(self.smoker_lineEdit.text()),
            "cigsPerDay": int(self.cigspd_lineEdit.text()),
            "BPMeds": int(self.BPmeds_lineEdit.text()),
            "prevalentStroke": int(self.Pstrokes_lineEdit.text()),
            "prevalentHyp": int(self.Phyp_lineEdit.text()),
            "diabetes": int(self.Diabetes_lineEdit.text()),
            "totChol": int(self.Totchol_lineEdit.text()),
            "sysBP": int(self.sysbp_lineEdit.text()),
            "diaBP": int(self.diabp_lineEdit.text()),
            "BMI": int(self.BMI_lineEdit.text()),
            "heartRate": int(self.hrate_lineEdit.text()),
            "glucose": int(self.gluco_lineEdit.text()),
            # Extract data from other fields similarly
            # ...
        }
        
        # Convert input data to a DataFrame
        input_df = pd.DataFrame([input_data])
        
        # Assuming rf_model is your trained Random Forest model
        predicted_result = self.predictOutcome(input_df,rf)
        
        # Display the prediction result in a message box
        if predicted_result[0] == 1:
            QMessageBox.about(self, "Prediction Result", "The model predicts a high risk of heart disease.")
        else:
            QMessageBox.about(self, "Prediction Result", "The model predicts a low risk of heart disease.")



    def predictOutcome(self, input_data,model):
        # Load your trained Random Forest model
        # Replace this with your actual trained model
        predicted_outcome = model.predict(input_data)
        return predicted_outcome


class ParentGUI(QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('Parent GUI')
        self.setGeometry(100, 100, 400, 300)

        central_widget = QWidget()
        self.setCentralWidget(central_widget)

        layout = QVBoxLayout()
        central_widget.setLayout(layout)

        self.heart_button = QPushButton('Open Heart GUI', self)
        self.heart_button.clicked.connect(self.open_heart_gui)

        self.diabetes_button = QPushButton('Open Diabetes GUI', self)
        self.diabetes_button.clicked.connect(self.open_diabetes_gui)

        layout.addWidget(self.heart_button)
        layout.addWidget(self.diabetes_button)

    def open_heart_gui(self):
        self.heart_gui = HeartGUI()
        self.heart_gui.show()

    def open_diabetes_gui(self):
        self.diabetes_gui = DiabetesGUI()
        self.diabetes_gui.show()
        


if __name__ == '__main__':
    app = QApplication(sys.argv)
    main_window = MainWindow()
    main_window.setFixedHeight(400)
    main_window.setFixedWidth(300)
    main_window.show()
    sys.exit(app.exec_())
